# On-Disk Concatenation of AnnData Files

**Author:** Selman Özleyen


## Initializing

Let's begin by importing the necessary libraries and modules. This notebook also uses the [memory-profiler](https://pypi.org/project/memory-profiler/) extension. Ensure you've installed it using `pip install memory-profiler` before proceeding.

This notebook uses the [memory-profiler](https://pypi.org/project/memory-profiler/) extension, call `pip install memory-profiler` before running this notebook.

In [1]:
from memory_profiler import memory_usage
import numpy as np
from scipy import sparse
import pandas as pd
import shutil
from anndata.tests.helpers import gen_typed_df
from anndata.experimental import write_elem
import zarr
import anndata
from pathlib import Path
import glob

import anndata
import zarr
import gc
from anndata.experimental import concat_on_disk
from dask.distributed import Client, LocalCluster

## Data Creation and Analysis

In this section, we'll demonstrate the core functionality of the `concat_on_disk` method. We'll create datasets and analyze how this method performs in terms of memory usage. This will help us understand its efficiency and benefits, especially when working with large datasets.

We will define parameters that will influence the structure of our datasets:

- **Shapes**: Defines the shape of array (e.g., "fat", "tall", "square").
- **Sizes**: The size of the array, indicating the number of elements.
- **Densities**: Specifies the data density. 1 means dense numpy array.

These parameters will be utilized in subsequent sections to generate and analyze datasets.

In [2]:

# Directory where the data will be stored
OUTDIR = Path("tmpdata")

# Parameters that will influence the structure and size of our datasets:

# Shapes of the arrays: "fat", "tall", or "square"
shapes = ["fat", "tall", "square"]

# Sizes of the dataset, indicating the number of elements
sizes = [10_000]

# Densities: Specifies the data density. A higher value means more non-zero elements
densities = [0.1, 1]

# Number of times each array type will be created
num_runs = 3


### create_adata

This function is designed to create an `AnnData` object, which is a foundational data structure used in bioinformatics to store high-dimensional data such as gene expression matrices. Given a data matrix `X` and its shape, the function constructs the `AnnData` object complete with observation (`obs`) and variable (`var`) metadata.

- `shape`: The shape (dimensions) of the data matrix.
- `X`: The actual data matrix (could be dense or sparse).

Returns: An `AnnData` object constructed from the input data and metadata.


In [3]:
def create_adata(shape, X):
    # Shape of the data matrix
    M, N = shape
    
    # Generating observation and variable names
    obs_names = pd.Index(f"cell{i}" for i in range(shape[0]))
    var_names = pd.Index(f"gene{i}" for i in range(shape[1]))
    
    # Creating observation and variable dataframes
    obs = gen_typed_df(M, obs_names)
    var = gen_typed_df(N, var_names)
    
    # Renaming columns to ensure uniqueness
    obs.rename(columns=dict(cat="obs_cat"), inplace=True)
    var.rename(columns=dict(cat="var_cat"), inplace=True)
    
    # Constructing the AnnData object
    adata = anndata.AnnData(X, obs=obs, var=var)
    adata.var_names_make_unique()
    adata.obs_names_make_unique()

    return adata


### generate_array_funcs_and_names

This function determines the type of array functions and their corresponding names based on the provided `density` parameter. It essentially helps in figuring out if the dataset is dense or sparse.

- `density`: The density of the dataset. If the density is 1, the dataset is dense; otherwise, it's sparse.

Returns: A tuple containing the list of array functions and their corresponding names.


In [4]:
def generate_array_funcs_and_names(density):
    array_funcs = []  # List to hold array functions
    array_names = []  # List to hold array names
    
    # Check if dataset is dense
    if density == 1:
        array_names.append("np")
        array_funcs.append(lambda x: x.toarray())
    else:
        # For sparse datasets, consider both csc and csr formats
        array_names.extend(["csc", "csr"])
        array_funcs.extend([sparse.csc_matrix, sparse.csr_matrix])
    
    return array_funcs, array_names

### generate_dimensions

Given a shape description (like "fat", "tall", or "square") and a base size, this function computes the exact dimensions \(M\) and \(N\) of the dataset. 

- `shape`: Description of the desired shape of the dataset.
- `size`: Base size for the dataset.

Returns: The dimensions \(M\) and \(N\) of the dataset.


In [5]:
def generate_dimensions(shape, size):
    # Default dimensions
    M = size
    N = size
    
    # If the shape isn't square, adjust the dimensions
    if shape != "square":
        other_size = size + int(size * np.random.uniform(0.2, 0.4))
        if shape == "fat":
            M = other_size
        elif shape == "tall":
            N = other_size
            
    return M, N


## Writing The Arrays To Disk

We will use the functions defined below to write the anndatas. There is no need to understand them all. However, the functions are also explained below for users who would like to create their own datasets to do the measurements.

### Functions Overview

#### 1. `write_data_to_zarr`

This function is responsible for writing a given dataset `X` to a Zarr format file. Zarr is a format for the storage of chunked, compressed, N-dimensional arrays, which is useful for efficient on-disk storage and retrieval of large datasets.

- **Parameters**:
    - `X`: The dataset to be written.
    - `shape`: Descriptive shape of the dataset.
    - `array_name`: Name representing the type of array (e.g., "np", "csc", "csr").
    - `outdir`: Directory where the Zarr file should be stored.
    - `file_id`: Identifier for the file, used in naming.

- **Returns**: A string report detailing the writing operation.

#### 2. `write_temp_data`

This function is designed to write temporary data based on the specified parameters to the output directory. It iteratively generates data sets based on shapes, sizes, densities, and number of runs, and writes each dataset to a Zarr format file using the `write_data_to_zarr` function.

- **Parameters**:
    - `shapes`: List of dataset shapes (e.g., "fat", "tall", "square").
    - `sizes`: List of dataset sizes.
    - `densities`: List of dataset densities.
    - `num_runs`: Number of iterations for data generation.
    - `outdir`: Directory where the Zarr files should be stored.
    - `rewrite`: Boolean flag; if True, any existing data in the output directory will be overwritten.

This function not only writes the datasets but also maintains a log of the datasets written in a file named "done.txt".




In [6]:
def write_data_to_zarr(X, shape, array_name, outdir, file_id):
    fname = str(outdir) + f"/{file_id:02d}_{shape}_{array_name}"
    adata = create_adata((X.shape[0], X.shape[1]), X)
    output_zarr_path = f"{str(fname)}.zarr"
    z = zarr.open_group(output_zarr_path)
    write_elem(z, "/", adata)
    zarr.consolidate_metadata(z.store)
    return f"wrote {X.shape[0]}x{X.shape[1]}_{array_name} -> {fname}\n"

def write_temp_data(shapes, sizes, densities, num_runs, outdir, rewrite=False):
    outdir.mkdir(exist_ok=True)
    if rewrite:
        (outdir / "done.txt").unlink(missing_ok=True)
    if (outdir / "done.txt").exists():
        print("already done")
        with open(outdir / "done.txt", "r") as f:
            for line in f.readlines():
                print(line)
        return

    saved = []
    file_id = 1
    for _ in range(num_runs):
        for shape in shapes:
            for size in sizes:
                for density in densities:
                    array_funcs, array_names = generate_array_funcs_and_names(density)
                    M, N = generate_dimensions(shape, size)

                    X_base = sparse.random(M, N, density=density, format="csc")

                    for array_func, array_name in zip(array_funcs, array_names):
                        X = array_func(X_base)
                        report = write_data_to_zarr(X, shape, array_name, outdir, file_id)
                        print(report)
                        saved.append(report)
                        file_id += 1
    with open(outdir / "done.txt", "w") as f:
        f.writelines(saved)



In [7]:

# You can call the function like this:
write_temp_data(shapes, sizes, densities, num_runs, OUTDIR)


already done
wrote 12941x10000_csc -> tmpdata/01_fat_csc

wrote 12941x10000_csr -> tmpdata/02_fat_csr

wrote 12289x10000_np -> tmpdata/03_fat_np

wrote 10000x12366_csc -> tmpdata/04_tall_csc

wrote 10000x12366_csr -> tmpdata/05_tall_csr

wrote 10000x13624_np -> tmpdata/06_tall_np

wrote 10000x10000_csc -> tmpdata/07_square_csc

wrote 10000x10000_csr -> tmpdata/08_square_csr

wrote 10000x10000_np -> tmpdata/09_square_np

wrote 13924x10000_csc -> tmpdata/10_fat_csc

wrote 13924x10000_csr -> tmpdata/11_fat_csr

wrote 13321x10000_np -> tmpdata/12_fat_np

wrote 10000x12377_csc -> tmpdata/13_tall_csc

wrote 10000x12377_csr -> tmpdata/14_tall_csr

wrote 10000x12595_np -> tmpdata/15_tall_np

wrote 10000x10000_csc -> tmpdata/16_square_csc

wrote 10000x10000_csr -> tmpdata/17_square_csr

wrote 10000x10000_np -> tmpdata/18_square_np

wrote 13778x10000_csc -> tmpdata/19_fat_csc

wrote 13778x10000_csr -> tmpdata/20_fat_csr

wrote 12484x10000_np -> tmpdata/21_fat_np

wrote 10000x13293_csc -> tmpdata

### Putting our arrays in categories

The `create_datasets` function constructs a dictionary that maps dataset types (dense or sparse) and their axis (0 or 1) to a set of corresponding file paths. The function processes different file sets and, based on conditions like `requires_reindexing`, refines the set of file paths to be associated with each dataset type and axis combination.


In [8]:
# files by properties
filesets = {
    'nps' : set(glob.glob(str(OUTDIR) + "/*np*")),
    'csrs' : set(glob.glob(str(OUTDIR) + "/*csr*")),
    'cscs' : set(glob.glob(str(OUTDIR) + "/*csc*")),
    'fats' : set(glob.glob(str(OUTDIR) + "/*fat*")),
    'talls' : set(glob.glob(str(OUTDIR) + "/*tall*")),
    'squares' :set(glob.glob(str(OUTDIR) + "/*square*")),
}

In [9]:
def create_datasets(filesets, requires_reindexing=False):
    data = dict()
    for fileset, axis in (("cscs",1), ("csrs",0), ("nps",0), ("nps",1)):
        filepaths = filesets[fileset].copy()
        if not requires_reindexing:
            tall_or_fat = filesets['talls'] if axis == 1 else filesets['fats']
            filepaths = filepaths.intersection(tall_or_fat.union(filesets['squares']))
        fileset_name = "dense" if fileset == "nps" else "sparse"
        data[fileset_name, axis] = filepaths
    return data    

Below you can see the both the list of anndatas that would require reindexing when concatenating (i.e, their axis size don't match) and the ones who don't

In [10]:
datasets_aligned, datasets_unaligned =create_datasets(filesets,requires_reindexing=False), create_datasets(filesets, requires_reindexing=True)

## Measuring Performance

### `get_arr_sizes`

This function calculates the size of the data arrays for a list of given file paths. It can accommodate both sparse and dense formats, adjusting the computation method accordingly.

---

### `get_mem_usage`

The function `get_mem_usage` evaluates the memory usage when performing on-disk concatenation using the `concat_on_disk` method. Depending on whether the dataset is sparse or dense, it either initiates a Dask cluster to handle the data or directly concatenates it. It returns the memory increment, the maximum memory used, the memory usage over time, and the initial memory.

---

### `dataset_max_mem`

The `dataset_max_mem` function profiles and prints the maximum memory usage when concatenating datasets of different types (sparse or dense) and along different axes. For each dataset and axis combination, it determines the files to concatenate, calculates their sizes, and then measures the memory usage during the concatenation process. The results are stored in a dictionary that maps the dataset type and axis to the corresponding memory usage metrics.


In [11]:
def get_arr_sizes(filepaths, is_sparse):
    def get_arr_size(g):
        if is_sparse:
            size = (
                g.store.getsize("X/data")
                + g.store.getsize("X/indices")
                + g.store.getsize("X/indptr")
            )
        else:
            size = g.store.getsize("X")
        return size

    return [get_arr_size(zarr.open_group(filepath)) for filepath in filepaths]


def get_mem_usage(filepaths, writepth, axis, max_arg, is_sparse):
    concat_kwargs = {
        "in_files": filepaths,
        "out_file": writepth,
        "axis": axis,
    }

    if not is_sparse:
        cluster = LocalCluster(n_workers=1, threads_per_worker=1, memory_limit=max_arg)
        client = Client(cluster)
    else:
        concat_kwargs["max_loaded_elems"] = max_arg

    # get the current memory usage
    initial_mem = memory_usage(-1, interval=0.001)[0]

    mem_usages = memory_usage(
        (
            concat_on_disk,
            (),
            concat_kwargs,
        ),
        include_children=True,
        interval=0.001,
    )
    max_mem = max(mem_usages)
    mem_increment = max_mem - initial_mem

    if not is_sparse:
        client.close()
        cluster.close()
    return mem_increment, max_mem, mem_usages, initial_mem


def dataset_max_mem(max_arg, datasets, array_type):
    results = {}
    is_sparse = array_type == "sparse"
    for filepaths,axis in [(datasets[array_type,axis],axis) for axis in [0,1]]:
        writepth = OUTDIR / f"{array_type}_{axis}.zarr"
        if writepth.exists():
            shutil.rmtree(writepth)

        # print the files we are concatenating
        print("Dataset:", array_type, axis)
        print(f"Concatenating {len(filepaths)} files with sizes:")
        sizes = get_arr_sizes(filepaths, is_sparse)
        print([str(s//(2**20))+'MiB' for s in sizes])
        print(f"Total size: {sum(sizes)//(2**20)}MiB")
        


        # force garbage collection
        gc.collect()
        # perform profiling
        mem_increment, max_mem, mem_usages, initial_mem = get_mem_usage(filepaths, writepth, axis, max_arg, is_sparse)
        # force garbage collection again
        gc.collect()

        print("Concatenation finished")
        print("Max memory increase:", int(mem_increment), "MiB")
        print("--------------------------------------------------")
        results[array_type, axis] = {"max_mem": max_mem, "increment": mem_increment}
    return results


## Results of concatenation without reindexing

In this section, we evaluate the memory performance of the `concat_on_disk` function when concatenating datasets **without** the need for reindexing. The printed reports provide details about the individual file sizes, the total dataset size, and the maximum memory increment during the concatenation.


### Sparse Datasets

For sparse datasets:

- We can observe that the function has been called multiple times with different memory constraints (`max_arg` values), and each time the datasets were concatenated successfully.
- It's crucial to note that even when the combined size of the files exceeds the allocated memory, the concatenation still proceeds efficiently. This behavior highlights the primary advantage of the `concat_on_disk` function: it performs the concatenation **on disk**, ensuring that memory consumption remains low, even for large datasets.
  
However, it's also worth noting that if one has sufficient memory to fit the files, performing the concatenation in memory would be faster.

### Dense Datasets

The results for dense datasets follow a similar pattern:

- The datasets are concatenated successfully under memory constraints.
- The total size of the dataset is much larger than the memory increment, reinforcing the efficiency of on-disk concatenation.


In [12]:
dataset_max_mem(max_arg=1_000_000_000, datasets=datasets_aligned, array_type='sparse')

Dataset: sparse 0
Concatenating 6 files with sizes:
['109MiB', '101MiB', '78MiB', '108MiB', '78MiB', '78MiB']
Total size: 554MiB
Concatenation finished
Max memory increase: 160 MiB
--------------------------------------------------
Dataset: sparse 1
Concatenating 6 files with sizes:
['78MiB', '78MiB', '78MiB', '97MiB', '97MiB', '104MiB']
Total size: 534MiB
Concatenation finished
Max memory increase: 159 MiB
--------------------------------------------------


{('sparse', 0): {'max_mem': 354.98046875, 'increment': 160.40625},
 ('sparse', 1): {'max_mem': 370.18359375, 'increment': 159.203125}}

In [13]:
dataset_max_mem(max_arg="1000MiB", datasets=datasets_aligned, array_type='dense')

Dataset: dense 0
Concatenating 6 files with sizes:
['891MiB', '668MiB', '835MiB', '668MiB', '822MiB', '668MiB']
Total size: 4556MiB
Concatenation finished
Max memory increase: 861 MiB
--------------------------------------------------
Dataset: dense 1
Concatenating 6 files with sizes:
['910MiB', '668MiB', '843MiB', '668MiB', '668MiB', '923MiB']
Total size: 4684MiB
Concatenation finished
Max memory increase: 1064 MiB
--------------------------------------------------


{('dense', 0): {'max_mem': 1084.64453125, 'increment': 861.359375},
 ('dense', 1): {'max_mem': 1295.5859375, 'increment': 1064.3671875}}

## Results of concatenation with reindexing

This section presents the results of the `concat_on_disk` function when concatenating datasets that **require** reindexing.

The observations and interpretations for this section are similar to the ones mentioned for the "without reindexing" section. The primary difference is the datasets used for the concatenation. Once again, the on-disk concatenation allows for efficient memory usage, even when the datasets need reindexing.

One can also see the effect of the memory contrain on the measurements.

In [14]:
dataset_max_mem(max_arg=1_000_000_000, datasets=datasets_unaligned, array_type='sparse')

Dataset: sparse 0
Concatenating 9 files with sizes:
['97MiB', '104MiB', '109MiB', '101MiB', '78MiB', '108MiB', '78MiB', '97MiB', '78MiB']
Total size: 853MiB
Concatenation finished
Max memory increase: 467 MiB
--------------------------------------------------
Dataset: sparse 1
Concatenating 9 files with sizes:
['109MiB', '101MiB', '78MiB', '78MiB', '78MiB', '97MiB', '97MiB', '108MiB', '104MiB']
Total size: 853MiB
Concatenation finished
Max memory increase: 631 MiB
--------------------------------------------------


{('sparse', 0): {'max_mem': 705.9765625, 'increment': 467.0078125},
 ('sparse', 1): {'max_mem': 879.07421875, 'increment': 631.796875}}

In [15]:
dataset_max_mem(max_arg=1_000_000, datasets=datasets_unaligned, array_type='sparse')

Dataset: sparse 0
Concatenating 9 files with sizes:
['97MiB', '104MiB', '109MiB', '101MiB', '78MiB', '108MiB', '78MiB', '97MiB', '78MiB']
Total size: 853MiB
Concatenation finished
Max memory increase: 204 MiB
--------------------------------------------------
Dataset: sparse 1
Concatenating 9 files with sizes:
['109MiB', '101MiB', '78MiB', '78MiB', '78MiB', '97MiB', '97MiB', '108MiB', '104MiB']
Total size: 853MiB
Concatenation finished
Max memory increase: 205 MiB
--------------------------------------------------


{('sparse', 0): {'max_mem': 455.08984375, 'increment': 204.109375},
 ('sparse', 1): {'max_mem': 458.26953125, 'increment': 205.38671875}}

In [16]:
dataset_max_mem(max_arg="1000MiB", datasets=datasets_unaligned, array_type='dense')

Dataset: dense 0
Concatenating 9 files with sizes:
['891MiB', '910MiB', '668MiB', '923MiB', '835MiB', '668MiB', '843MiB', '822MiB', '668MiB']
Total size: 7233MiB
Concatenation finished
Max memory increase: 1056 MiB
--------------------------------------------------
Dataset: dense 1
Concatenating 9 files with sizes:
['891MiB', '910MiB', '668MiB', '923MiB', '835MiB', '668MiB', '843MiB', '822MiB', '668MiB']
Total size: 7233MiB
Concatenation finished
Max memory increase: 969 MiB
--------------------------------------------------


{('dense', 0): {'max_mem': 1308.2265625, 'increment': 1056.375},
 ('dense', 1): {'max_mem': 1226.05078125, 'increment': 969.38671875}}